## JetRacer Calibration

Due to the different architecture, your JetRacer may have a reverse steering and/or throttle.

Here you can use this notebook to calibrate your JetRacer.

In [1]:
import ipywidgets
import threading
import time
import traitlets
import sys
sys.path.append('../')
from robot.jetracer import JetRacer

robot = JetRacer(bus=7, signal_freq=50, servo_channel=0, motor_channel=1)

In [2]:
robot.steering = 0.0

Now we will command the JetRacer to do some basic movement, make sure your JetRacer is not touching any surface and switch the control to PC mode when you ready

In [3]:
jetracer_running = True

def jetracer_test0():
    count = 0.0
    delta = 0.01
    while jetracer_running:
        if count > 1.0 or count < -1.0:
            delta = -delta
        count += delta
        robot.steering = count
        robot.throttle = count / 2
        time.sleep(0.1)
        
jetracer_test0_thread = threading.Thread(target=jetracer_test0)
jetracer_test0_thread.start()

If your JetRacer is not swinging its front steering and having a changing throttle, make sure your motor power is turned on and the PC mode LED is on.

After checking all the hardware is working fine, we can command the robot to stop

In [4]:
jetracer_running = False
jetracer_test0_thread.join()
robot.stop()

Here we will calibrate the front steering of our JetRacer.

We want to make sure that we utilize the full potential of our JetRacer's servo and it's moving straight when we set `robot.steering = 0`.

Drag the pulse width slider to find the minimum and maximum pulse width, click the button "Minimum", "Center", "Maximum" when the servo reaches its corresponding position.

The unit of the pulse width slider is in microsecond.

In [5]:
min_pulse_width = 1000
center_pulse_width = 1500
max_pulse_width = 2000

pulse_width_slider = ipywidgets.IntSlider(min=500, max=2500, step=1, value=1500, description='Pulse Width')
min_button = ipywidgets.Button(description="Minimum")
center_button = ipywidgets.Button(description="Center")
max_button = ipywidgets.Button(description="Maximum")
buttons_hbox = ipywidgets.HBox([min_button, center_button, max_button])

def on_pulse_width_change(change):
    period = 1000000 / robot.pca.frequency
    robot.pca[robot.servo.channel] = change['new'] / period

def on_min_clicked(b):
    min_pulse_width = pulse_width_slider.value
    robot.servo._cal_alpha_beta(robot.pca.frequency, min_pulse_width, center_pulse_width, max_pulse_width)
    print(f"Minimum pulse width set to {min_pulse_width} us")
    
def on_center_clicked(b):
    center_pluse_width = pulse_width_slider.value
    robot.servo._cal_alpha_beta(robot.pca.frequency, min_pulse_width, center_pulse_width, max_pulse_width)
    print(f"Center pulse width set to {center_pulse_width} us")
    
def on_max_clicked(b):
    max_pulse_width = pulse_width_slider.value
    robot.servo._cal_alpha_beta(robot.pca.frequency, min_pulse_width, center_pulse_width, max_pulse_width)
    print(f"Maximum pulse width set to {max_pulse_width} us")

pulse_width_slider.observe(on_pulse_width_change, names='value')
min_button.on_click(on_min_clicked)
center_button.on_click(on_center_clicked)
max_button.on_click(on_max_clicked)        

display(ipywidgets.VBox([pulse_width_slider, buttons_hbox]))

And we still need to make sure the front steering is at the leftmost when `robot.steering = -1.0`, rightmost when `robot.steering = 1.0`.

Click the button reverse when you see the direction of the front steering is reversed.